In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install -q transformers datasets accelerate peft bitsandbytes trl sentencepiece

In [ ]:
!pip install -U \
  transformers==4.40.2 \
  peft==0.11.1 \
  accelerate==0.30.1 \
  bitsandbytes==0.43.1 \
  datasets==2.16.1 \
  fsspec==2023.6.0 \
  gcsfs==2023.6.0

  Using cached transformers-4.40.2-py3-none-any.whl.metadata (137 kB)
  Using cached peft-0.11.1-py3-none-any.whl.metadata (13 kB)
  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl.metadata (2.2 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached pyarrow_hotfix-0.7-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB

In [ ]:
import os
import json
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# MODEL_NAME = "google/gemma-2b-it"  # Use gemma-4b-it if you have >24GB GPU (QLoRA recommended for that)
os.environ['HF_TOKEN']="" # add the hugging face token
MODEL_NAME = "google/flan-t5-large"
DATASET_DIR = "/content/drive/MyDrive/DL_project/data/references/train"
MAX_LEN = 1024

In [ ]:
def load_jsonl_dataset(data_dir):
    all_data = []
    for lang_pair in os.listdir(data_dir):
        lang_dir = os.path.join(data_dir, lang_pair)
        train_path = os.path.join(lang_dir, "train.jsonl")
        if os.path.exists(train_path):
            with open(train_path, 'r', encoding='utf-8') as f:
                records = [json.loads(line) for line in f]
                all_data.extend(records)
    return Dataset.from_list(all_data)

In [ ]:
raw_dataset = load_jsonl_dataset(DATASET_DIR)

In [ ]:
def format_prompt(example):
    return {
        "text": f"### Source ({example['source_locale']}):\n{example['source']}\n\n### Translation ({example['target_locale']}):\n{example['target']}"
    }

formatted_dataset = raw_dataset.map(format_prompt)


Map:   0%|          | 0/32962 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token


def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/32962 [00:00<?, ? examples/s]

In [ ]:
# for gemma
# def load_model(full_finetune=True):
#     model = AutoModelForCausalLM.from_pretrained(
#         MODEL_NAME,
#         device_map="auto",
#         load_in_4bit=not full_finetune,
#         torch_dtype=torch.float16,
#         trust_remote_code=True
#     )

#     if full_finetune:
#         return model

#     model = prepare_model_for_kbit_training(model)

#     lora_config = LoraConfig(
#         r=8,
#         lora_alpha=16,
#         target_modules=["q_proj", "v_proj"],
#         lora_dropout=0.05,
#         bias="none",
#         task_type="CAUSAL_LM"
#     )

#     return get_peft_model(model, lora_config)


In [ ]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import get_peft_model, prepare_model_for_kbit_training

from peft import LoraConfig
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],  # or adjust based on your model
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

def load_model(full_finetune=True):
    model_name = MODEL_NAME  # Make sure this global variable is set

    # Choose model class based on architecture
    if "t5" in model_name or "flan" in model_name:
        model_class = AutoModelForSeq2SeqLM
    else:
        model_class = AutoModelForCausalLM

    if full_finetune:
        # Full fine-tuning (no quantization, all parameters trainable)
        model = model_class.from_pretrained(model_name, device_map="auto")

    else:
        # QLoRA (4-bit quantization with LoRA adapters)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )

        model = model_class.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto"
        )

        # Prepare for LoRA training
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    return model


In [ ]:
model = load_model(full_finetune=True)  # for full finetuning
# model = load_model(full_finetune=False)  # for LoRA

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    evaluation_strategy="no",
    learning_rate=2e-5,
    fp16=True,
    gradient_checkpointing=True,
    report_to="none"
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:479: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.084800
70,0.000000
80,0.000000
90,0.000000
100,0.000000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

In [ ]:
model.save_pretrained("/content/drive/MyDrive/DL_project/flan_t5_large_finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/DL_project/flan_t5_large_finetuned")